In [1]:
import psycopg2
import json
import pandas as pd
import requests

/Users/deel/Desktop/work/virtual_environments/payment_reconciliation_env/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
with open('/Users/deel/.Credentials/login_details.json', 'r') as fp:
    credentials = json.load(fp)

In [7]:
con = psycopg2.connect(dbname=credentials['database'], user=credentials['username'], password=credentials['password'], host=credentials['host'])

In [ ]:
# recommender = get_invoice_recommendations_locally(143103)

In [8]:
last_2_months_query = """select i.invoice_id, i.payments_transaction_id 
from "public".invoice_payments_transactions as i JOIN payments."Transaction"as t 
on i.payments_transaction_id=t.id
where t.created_at>'03/01/2022'"""

In [10]:
last_2_months_df = pd.read_sql(last_2_months_query, con)

In [12]:
last_2_months_df.shape

(12027, 2)

In [13]:
last_2_months_df.head(5)


,invoice_id,payments_transaction_id
0,2484631,6887851
1,2473682,6984974
2,2472973,6984967
3,2484657,6995201
4,2453291,6992951


In [84]:
payload = {'transactionId': str(6887851), 'invoiceId': str(2484631)}

In [85]:
response = requests.post(url, json = payload).json()

In [87]:
response = response[0]
response['transaction_id'] = 6887851

In [88]:
response


{'amount_difference': 0.0,
 'company_name_match_ratio': 1.0,
 'days_until_money_received': 0,
 'deel_ref_extracted': True,
 'deel_ref_match': True,
 'id': 2484631,
 'match_score': 1,
 'rel_abs_amount_difference': 0.0,
 'rel_amount_difference': 0.0,
 'transaction_amount': 9754.98,
 'transaction_deel_reference': 'DEELJXCGYJ',
 'transaction_id': 6887851}

In [20]:
from time import time

In [22]:
time()

1651502110.920141

In [95]:
start = time()
url = 'http://127.0.0.1:5000//get-transaction-invoice-comparison-locally'
correct_matches = []
faulty = []
for i, row in last_2_months_df[10700:].iterrows():
#     if i>1500:
#         break
    if not i%100:
        print(f"time elapsed to {i}: {time()-start}")
    if not i%200:
        df = pd.DataFrame(correct_matches)
        df.to_csv(f"correct_matches_{i-200}-{i}.csv")
        correct_matches = []
    payload = {'transactionId': str(row['payments_transaction_id']), 'invoiceId': str(row['invoice_id'])}
    response = requests.post(url, json = payload).json()
    if response:
        response = response[0]
        response['transaction_id'] = row['payments_transaction_id']
        correct_matches.append(response)
    else:
        faulty.append(payload)
stop = time()
print(f"runtime: {stop-start}")

time elapsed to 10700: 0.0005700588226318359
time elapsed to 10800: 189.23489499092102
time elapsed to 10900: 375.4224512577057
time elapsed to 11000: 561.7206292152405
time elapsed to 11100: 749.3328821659088
time elapsed to 11200: 937.1337840557098
time elapsed to 11300: 1123.5951552391052
time elapsed to 11400: 1309.8675429821014
time elapsed to 11500: 1495.7068519592285
time elapsed to 11600: 1684.2333221435547
time elapsed to 11700: 1872.2299370765686
time elapsed to 11800: 2063.812577009201
time elapsed to 11900: 2252.3261439800262
time elapsed to 12000: 2439.403528213501
runtime: 2490.065262079239


In [103]:
len(correct_matches)

27

In [104]:
df = pd.DataFrame(correct_matches)
df.to_csv(f"correct_matches_{1227-200}-{1227}.csv")

In [105]:
import os

In [106]:
files = [file for file in os.listdir() if file.endswith('.csv')]

In [107]:
for i, file in enumerate(files):
    if not i:
        print('not i')
        overall_df = pd.read_csv(file)
    else:
        print(i)
        overall_df = overall_df.append(pd.read_csv(file))

not i
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68


## Perfect matches

In [108]:
perfect_matches = overall_df[overall_df['match_score']==1]

In [109]:
perfect_matches.shape[0]/overall_df.shape[0]

,Unnamed: 0,amount_difference,company_name_match_ratio,days_until_money_received,deel_ref_extracted,deel_ref_match,id,match_score,rel_abs_amount_difference,rel_amount_difference,transaction_amount,transaction_deel_reference,transaction_id
0,0,0.0,1.00,0.0,True,True,3010883.0,1.0,0.0,0.0,11460.72,DEELKKVFJU,12227411.0
1,1,0.0,1.00,0.0,True,True,3012984.0,1.0,0.0,0.0,6535.90,DEELUVVHBK,12219739.0
2,2,0.0,1.00,0.0,True,True,3011198.0,1.0,0.0,0.0,77553.21,DEELJWQ8LA,12227417.0
3,3,0.0,1.00,0.0,True,True,3013179.0,1.0,0.0,0.0,3921.45,DEELF4VEEB,12219733.0
4,4,0.0,1.00,0.0,True,True,3010893.0,1.0,0.0,0.0,24833.73,DEELPW7RHZ,12227419.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,0.0,0.12,0.0,True,True,3177333.0,1.0,0.0,0.0,19747.73,DEEL5Z9PS9,13120790.0
196,196,0.0,0.00,0.0,True,True,3177417.0,1.0,0.0,0.0,16015.48,DEELFHJ9JH,13120791.0
197,197,0.0,0.80,2.0,True,True,3049931.0,1.0,0.0,0.0,8234.13,DEELNA8ZUG,13121047.0
198,198,0.0,1.00,0.0,True,True,3176615.0,1.0,0.0,0.0,8467.98,DEELSEAQJX,13121042.0


In [165]:
perfect_matches.shape[0]/overall_df.shape[0]

0.7050850311500253

## What about when the top match score isn't 100% but is still the correct match?

In [110]:
imperfect_matches = overall_df[overall_df['match_score']!=1]

In [111]:
imperfect_matches

,Unnamed: 0,amount_difference,company_name_match_ratio,days_until_money_received,deel_ref_extracted,deel_ref_match,id,match_score,rel_abs_amount_difference,rel_amount_difference,transaction_amount,transaction_deel_reference,transaction_id
71,71,-15.00,1.0,0.0,True,True,3013234.0,0.999963,0.000197,-0.000197,76152.46,DEELQMCAAL,12223166.0
79,79,-4.72,1.0,0.0,True,True,3015421.0,0.999963,0.000195,-0.000195,24152.73,DEEL3F64KG,12230871.0
80,80,15.00,1.0,0.0,True,True,3012936.0,0.999587,0.002205,0.002205,6816.70,DEELJE476Z,12223164.0
84,84,-10.00,0.3,1.0,True,True,2966184.0,0.743423,0.001746,-0.001746,5718.58,DEEL4XNCP8,11929841.0
86,86,-0.54,1.0,-4.0,True,False,3012504.0,0.500000,0.015670,-0.015670,33.92,DEELKVL62D,11955976.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,190,0.03,1.0,2.0,False,False,3046749.0,0.729165,0.000008,0.000008,3623.25,NaN,13098082.0
191,191,-4.73,1.0,5.0,False,False,3010920.0,0.666554,0.000603,-0.000603,7833.33,NaN,13094570.0
192,192,-4.71,1.0,5.0,False,False,3010933.0,0.666526,0.000750,-0.000750,6276.17,NaN,13094569.0
193,193,-0.01,1.0,9.0,False,False,2980553.0,0.666666,0.000003,-0.000003,3406.37,NaN,13098077.0


In [113]:
overly_perfect_matches = overall_df[overall_df['match_score']>1]

In [116]:
imperfect_matches.reset_index(inplace=True)

In [117]:
imperfect_matches

,index,Unnamed: 0,amount_difference,company_name_match_ratio,days_until_money_received,deel_ref_extracted,deel_ref_match,id,match_score,rel_abs_amount_difference,rel_amount_difference,transaction_amount,transaction_deel_reference,transaction_id
0,71,71,-15.00,1.0,0.0,True,True,3013234.0,0.999963,0.000197,-0.000197,76152.46,DEELQMCAAL,12223166.0
1,79,79,-4.72,1.0,0.0,True,True,3015421.0,0.999963,0.000195,-0.000195,24152.73,DEEL3F64KG,12230871.0
2,80,80,15.00,1.0,0.0,True,True,3012936.0,0.999587,0.002205,0.002205,6816.70,DEELJE476Z,12223164.0
3,84,84,-10.00,0.3,1.0,True,True,2966184.0,0.743423,0.001746,-0.001746,5718.58,DEEL4XNCP8,11929841.0
4,86,86,-0.54,1.0,-4.0,True,False,3012504.0,0.500000,0.015670,-0.015670,33.92,DEELKVL62D,11955976.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3498,190,190,0.03,1.0,2.0,False,False,3046749.0,0.729165,0.000008,0.000008,3623.25,NaN,13098082.0
3499,191,191,-4.73,1.0,5.0,False,False,3010920.0,0.666554,0.000603,-0.000603,7833.33,NaN,13094570.0
3500,192,192,-4.71,1.0,5.0,False,False,3010933.0,0.666526,0.000750,-0.000750,6276.17,NaN,13094569.0
3501,193,193,-0.01,1.0,9.0,False,False,2980553.0,0.666666,0.000003,-0.000003,3406.37,NaN,13098077.0


In [169]:
start = time()
recommendations = []
no_responses = []
url="http://127.0.0.1:5000//get-invoice-recommendations-locally"

for i, row in imperfect_matches[:2000].iterrows():
    
    payload = {'transactionId': row['transaction_id']}
    response = requests.post(url, json=payload)
    if response.json():
        recommendations.append({'transaction_id': row['transaction_id'], 'recommendations': response.json()})
    else:
        no_responses.append(row['transaction_id'])
    if not i%10:
        print(f"time elapsed to {i}: {time()-start}")

       
print(f"time elapsed to {i}: {time()-start}")   

time elapsed to 0: 27.667723655700684
time elapsed to 10: 282.1306927204132
time elapsed to 20: 687.8165838718414
time elapsed to 30: 903.5612909793854
time elapsed to 40: 1013.6203646659851
time elapsed to 50: 1245.291821718216
time elapsed to 60: 1416.9555866718292
time elapsed to 70: 1584.475917816162
time elapsed to 80: 1847.7701439857483
time elapsed to 90: 2035.7772648334503
time elapsed to 100: 2147.1192858219147
time elapsed to 110: 2401.5371747016907
time elapsed to 120: 2829.7965230941772
time elapsed to 130: 3171.2353937625885
time elapsed to 140: 3460.956422805786
time elapsed to 150: 3813.300304889679
time elapsed to 160: 4013.0631020069122
time elapsed to 170: 4207.538912773132
time elapsed to 180: 4449.899901866913
time elapsed to 190: 4655.782297849655
time elapsed to 200: 4843.294173955917
time elapsed to 210: 5091.191795825958
time elapsed to 220: 5500.548215866089
time elapsed to 230: 5815.044723987579
time elapsed to 240: 6114.106156826019
time elapsed to 250: 6395.

JSONDecodeError: [Errno Expecting value] <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>
: 0

In [170]:
recommendations[0]

{'transaction_id': 12223166.0,
 'recommendations': [{'amount_difference': -15.0,
   'company_name_match_ratio': 1.0,
   'days_until_money_received': 0,
   'deel_ref_extracted': True,
   'deel_ref_match': True,
   'id': 3013234,
   'match_score': 0.9999630747828534,
   'rel_abs_amount_difference': 0.00019693449144818532,
   'rel_amount_difference': -0.00019693449144818532,
   'transaction_amount': 76152.46,
   'transaction_deel_reference': 'DEELQMCAAL'},
  {'amount_difference': 1133.3300000000017,
   'company_name_match_ratio': 0.75,
   'days_until_money_received': -2,
   'deel_ref_extracted': True,
   'deel_ref_match': False,
   'id': 3027413,
   'match_score': 0.7315423975024771,
   'rel_abs_amount_difference': 0.015107213320122503,
   'rel_amount_difference': 0.015107213320122503,
   'transaction_amount': 76152.46,
   'transaction_deel_reference': 'DEELQMCAAL'},
  {'amount_difference': 937.5300000000134,
   'company_name_match_ratio': 0.29,
   'days_until_money_received': -1,
   'dee

In [171]:
recs = recommendations[1]['recommendations']

In [172]:
[(rec['id'], rec['match_score']) for rec in recs]

[(3015421, 0.9999633653386429),
 (3013086, 0.638223997637015),
 (3021601, 0.6190489354959258),
 (3009036, 0.6038023026732359),
 (2996558, 0.4865627410438807),
 (3011717, 0.419375),
 (3022272, 0.411875),
 (3022311, 0.374375),
 (3021437, 0.3725),
 (3001773, 0.370625)]

In [173]:
for recommendation in recommendations:
    counter_examples = False
    recs = recommendation['recommendations']
    id_and_score = [(rec['id'], rec['match_score']) for rec in recs]
    scores = [rec['match_score'] for rec in recs]
    if scores[0]==max(scores):
        pass
    else:
        counter_examples = True
        break

In [174]:
counter_examples

False

In [175]:
last_2_months_df

,invoice_id,payments_transaction_id
0,2484631,6887851
1,2473682,6984974
2,2472973,6984967
3,2484657,6995201
4,2453291,6992951
...,...,...
12022,3177722,13128467
12023,3054916,13129637
12024,3158420,13129649
12025,3177699,13129651


In [176]:
transaction_invoice_dic= {}

In [177]:
for _, row in last_2_months_df.iterrows():
    transaction_invoice_dic[row['payments_transaction_id']] = row['invoice_id']

In [178]:
transaction_invoice_dic

{6887851: 2484631,
 6984974: 2473682,
 6984967: 2472973,
 6995201: 2484657,
 6992951: 2453291,
 7000868: 2470099,
 7000869: 2471119,
 7000861: 2472114,
 7000872: 2470089,
 7000879: 2465947,
 7000880: 2470091,
 7000871: 2470785,
 7000878: 2452622,
 7000881: 2422877,
 7000882: 2472433,
 7000888: 2456635,
 7000889: 2470525,
 7000884: 2469653,
 7000892: 2471371,
 7000890: 2471981,
 7000894: 2470108,
 7000893: 2474537,
 7000895: 2472272,
 7000896: 2464861,
 7000900: 2469797,
 7000897: 2474069,
 7000902: 2470345,
 7000901: 2474299,
 7000898: 2471385,
 7000905: 2469776,
 7000914: 2455170,
 7000910: 2473357,
 7002183: 2488816,
 7000915: 2465744,
 7008559: 2466597,
 7008562: 2440317,
 7008574: 2454413,
 7008563: 2453267,
 7008580: 2470083,
 7008575: 2472169,
 7008576: 2467599,
 7013279: 2489555,
 7016170: 2490277,
 7017280: 2490247,
 7017278: 2490252,
 7017281: 2490244,
 7017285: 2490219,
 7017287: 2489583,
 7017283: 2490228,
 7020054: 2490253,
 7022950: 2479208,
 7022958: 2490337,
 7029588: 24

In [180]:
best_recommendations = {recommendation['transaction_id']: recommendation['recommendations'][0] for recommendation in recommendations}

In [181]:
recommendations

[{'transaction_id': 12223166.0,
  'recommendations': [{'amount_difference': -15.0,
    'company_name_match_ratio': 1.0,
    'days_until_money_received': 0,
    'deel_ref_extracted': True,
    'deel_ref_match': True,
    'id': 3013234,
    'match_score': 0.9999630747828534,
    'rel_abs_amount_difference': 0.00019693449144818532,
    'rel_amount_difference': -0.00019693449144818532,
    'transaction_amount': 76152.46,
    'transaction_deel_reference': 'DEELQMCAAL'},
   {'amount_difference': 1133.3300000000017,
    'company_name_match_ratio': 0.75,
    'days_until_money_received': -2,
    'deel_ref_extracted': True,
    'deel_ref_match': False,
    'id': 3027413,
    'match_score': 0.7315423975024771,
    'rel_abs_amount_difference': 0.015107213320122503,
    'rel_amount_difference': 0.015107213320122503,
    'transaction_amount': 76152.46,
    'transaction_deel_reference': 'DEELQMCAAL'},
   {'amount_difference': 937.5300000000134,
    'company_name_match_ratio': 0.29,
    'days_until_mo

In [182]:
best_algo_recommendations = {recommendation['transaction_id']: recommendation['recommendations'][0]['id'] for recommendation in recommendations}

In [183]:
best_algo_recommendations

{12223166.0: 3013234,
 12230871.0: 3015421,
 12223164.0: 3012936,
 11929841.0: 2966184,
 11955976.0: 2980645,
 12223527.0: 3013455,
 12097662.0: 3001508,
 12098505.0: 3000725,
 12110594.0: 3002584,
 11869736.0: 2973675,
 12115400.0: 3002673,
 12115418.0: 3002946,
 12236757.0: 3014501,
 12121358.0: 3004730,
 12133625.0: 3016443,
 12238731.0: 3015147,
 12239294.0: 3013765,
 12239295.0: 3013847,
 12236749.0: 3027395,
 12234765.0: 3012800,
 12234764.0: 3015770,
 12223920.0: 3014260,
 9210311.0: 2736409,
 9210315.0: 2690802,
 9210310.0: 2735097,
 9210308.0: 2715426,
 9210302.0: 2715106,
 9208204.0: 2716096,
 9208696.0: 2706111,
 9208687.0: 2706075,
 9208343.0: 2715447,
 9208685.0: 2710802,
 9206174.0: 2715397,
 9206175.0: 2715395,
 9206180.0: 2715346,
 9203536.0: 2688434,
 9233716.0: 2717028,
 9202365.0: 2713301,
 9203166.0: 2670979,
 7468946.0: 2586059,
 9199882.0: 2669637,
 9195405.0: 2731065,
 9241598.0: 2719421,
 9240195.0: 2721395,
 9240045.0: 2719728,
 9239628.0: 2721565,
 9239608.0: 

In [184]:
counter = 0
for key, value in best_algo_recommendations.items():
    if transaction_invoice_dic[key]==value:
        counter += 1
print(f'best algorithmic recommendation was correct in {counter*100/len(best_algo_recommendations)}% of cases')

best algorithmic recommendation was correct in 69.02356902356902% of cases


## So once we eliminate the perfect matches, the top picks are still 70% correct, putting us at 0.7 +0.3X0.7 = 91% matching correctly

In [190]:
ninety_five_percent_match = perfect_matches = overall_df[overall_df['match_score']>=0.95]

In [191]:
ninety_five_percent_match.shape

(8623, 13)

In [189]:
counter = 0
for key, value in second_best_algo_recommendations.items():
    if transaction_invoice_dic[key]==value:
        counter += 1
print(f'best algorithmic recommendation was correct in {counter*100/len(second_best_algo_recommendations)}% of cases')

best algorithmic recommendation was correct in 8.277027027027026% of cases


In [ ]:
matched_invoice_transaction_df = pd.read_sql("select * from public.invoice_payments_transactions", con)

In [ ]:
transaction_id = 7443022
transactions_query = f"""
        SELECT
            *
        FROM
            payments. "Transaction"
        WHERE
            id in ({str(transaction_id)})"""

transaction_df = pd.read_sql(transactions_query, con)

In [ ]:
transaction_df

In [ ]:
transaction_df.iloc[0]['data']

In [ ]:
invoice_id = 2469750
invoice_query = f"""
SELECT * 
FROM invoice
WHERE id IN ({str(invoice_id)})"""

In [ ]:
invoice_df = pd.read_sql(invoice_query, con)

In [ ]:
invoice_df

In [ ]:
for i, row in invoice_df.iterrows():
    for col in invoice_df.columns:
        print(col)
        print(row[col])
        print('************')

In [ ]:
transaction_df.iloc[0]['data']

In [ ]:
matched_invoice_transaction_df.head(10)

In [ ]:
transaction_df

In [ ]:
matched_invoice_transaction_df[matched_invoice_transaction_df['payments_transaction_id']==257382]

In [ ]:
transaction_id_list = [str(id) for id in matched_invoice_transaction_df['payments_transaction_id'][:20]]
transaction_string = ','.join(transaction_id_list)

In [ ]:
transactions = pd.read_sql(f'select * from payments. "Transaction" where id in ({5173879})', con)

In [ ]:
transactions

In [ ]:
invoices = pd.read_sql(f'select * from public.invoice where id in (2044669, 2044677, 2044683)', con)

In [ ]:
invoices


In [ ]:
invoices

In [ ]:
transaction = transactions.iloc[0]

In [ ]:
transaction['id']

In [ ]:
url="http://127.0.0.1:5000/get-examples-locally"


In [ ]:
transactions

In [ ]:
request_list = []
for i, transaction in transactions[:5].iterrows():

    payload = {'transactionId': str(transaction['id'])}
    response = requests.post(url, json=payload)
    request_list.append({'transaction': transaction['id'], 'response': response.json()})

In [ ]:
import requests
url="http://127.0.0.1:5000//get-invoice-recommendations-locally"
payload = {'transactionId': str(6597058)}
response = requests.post(url, json=payload)

In [ ]:
response.json()

In [ ]:
match_df = pd.read_sql("select * from invoice_payments_transactions where payments_transaction_id=6403097", con)

In [ ]:
match_df

In [ ]:
response_df = pd.DataFrame(response.json())

In [ ]:
response_df

In [ ]:
request_list[2]['response']

In [ ]:
for d in request_list[4]['response']:
    print(d['match_score'])

In [ ]:
for response  in request_list:
    print response['transactionId']
    print
    

In [ ]:
matched_invoice_transaction_df

In [ ]:
pd.read_sql(f'select * from payments. "Transaction" where id = 3031744', con)['data']

In [ ]:
pd.read_sql(f'select * from public. "invoice" where id = 1687151', con)

In [ ]:
transaction_3031744 = pd.read_sql('select * from payments. "Transaction" where id=3031744', con)

In [ ]:
transaction_3031744

In [ ]:
invoice_refs = pd.read_sql('select ref from public.invoice', con)

In [ ]:
def lower_str(x):
    if type(x)==type('hi'):
        return x.lower()
    return ''

In [ ]:
invoice_refs['lower_refs'] = invoice_refs['ref'].apply(lower_str)

In [ ]:
invoice_refs[invoice_refs['lower_refs']==str.lower('DEELMRWSF4')]

In [ ]:
str.lower('DEELMRWSF4')

In [ ]:
transaction_3031744.iloc[0].data

## investigating a particular transaction

In [10]:
payload_2 = {'transactionId': 122508}
url_2 = 'http://127.0.0.1:5000//get-invoice-recommendations-locally'
response_2 = requests.post(url_2, json = payload_2)

In [11]:
response_2.json()

[]

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(response_2.json())

In [ ]:
for recommendations we weren't pulling transfers made via coinbase, so all USDC payments were being skipped. Added coinbase to transfer methods in query

In [ ]:
get-transaction-invoice-comparison-locally
def compare_transaction_invoice_locally():
    data = request.json
    transaction_id = data['transactionId']
    invoice_id = data['invoiceId']

In [45]:
url = 'http://127.0.0.1:5000//get-transaction-invoice-comparison-locally'
payload = {'transactionId': 12761016, 'invoiceId': 3047744 }

In [50]:
response = requests.post(url, json = payload)

In [51]:
response.json()

[{'amount_difference': -13.80999999999949,
  'company_name_match_ratio': 1.0,
  'days_until_money_received': 0,
  'deel_ref_extracted': True,
  'deel_ref_match': True,
  'id': 3047744,
  'match_score': 0.9996544477569714,
  'rel_abs_amount_difference': 0.0018429452961523,
  'rel_amount_difference': -0.0018429452961523,
  'transaction_amount': 7479.63,
  'transaction_deel_reference': 'DEELESJ5KN'}]

In [13]:
transactions_query = f"""
            SELECT
                *
            FROM
                payments. "Transaction"
            WHERE
                id = 122508"""

In [19]:
transaction = pd.read_sql(transactions_query, con)

In [21]:
transaction.columns

Index(['id', 'provider', 'provider_ref', 'type', 'account', 'amount',
       'currency', 'description', 'data', 'created_at'],
      dtype='object')

In [22]:
transaction['data']

0    {'amount': 515, 'account': {'aba': '322271627'...
Name: data, dtype: object